## TexGen File Parsing and Processing
This section goes through all the preliminary steps necessary to proccess the export file from TexGen. You will need to edit the file names and dimensions in the 3rd code block.

In [ ]:
import pandas as pd
import numpy as np
import csv
from scipy.spatial import ConvexHull, Delaunay
import matplotlib.pyplot as plt
from scipy.optimize import linprog
import math

This is some stuff related to the scaling from TexGen to Macaw. Mostly just scratch work, but it does include NonDimConv which scales things later on.

In [ ]:
YarnHeight = 0.1
YarnWidth = 0.8

TowAreaInMicronSquared = 112257.84

ratio = YarnWidth/YarnHeight

TexGenToMicron = math.sqrt(TowAreaInMicronSquared/(math.pi *ratio))/(YarnHeight/2)
print(TexGenToMicron)
# 4644 * 120 = 557280
# NonDimConv1 = TexGenToMicron/557280
# print(1/NonDimConv1)

# NonDimConv = 4644 * TexGenToMicron
NonDimConv = 557280
print(NonDimConv)



Here we define a box the same size as our TexGen file. We also define the number of points that we want in each direction.

In [ ]:
InputFileName = 'ExampleFiber.inp'
OutputFileNameBase = 'ExampleFiber_'


x_max = 2.5 
x_min = -0.5
y_max = 0.5
y_min = -0.5
z_max = 0.21 
z_min = -0.01

x_dim = round(512 * (x_max - x_min))
y_dim = round(512 * (y_max - y_min))
z_dim = round(512 * (z_max - z_min))

x_step = (x_max - x_min)/x_dim
y_step = (y_max - y_min)/y_dim
if z_dim != 0: 
    z_step = (z_max - z_min)/z_dim

print(f"x_dim: {x_dim}, y_dim: {y_dim}, z_dim: {z_dim}")
print(f"x_step: {x_step}, y_step: {y_step}, z_step: {z_step}")


Using the box that we just defined, we can create a list of points so they are evenly spaced throughout the box. This is the basis of how an EBSD input file is meant to look. 

In [ ]:
x_start_value = x_min + x_step/2
y_start_value = y_min + y_step/2
z_start_value = z_min + z_step/2
print(f"x_start_value: {x_start_value}, y_start_value: {y_start_value}, z_start_value: {z_start_value}")

allpointslist = []

if z_dim == 0:
    print("2D file")
    y_value = y_start_value
    for _ in range(y_dim):
        x_value = x_start_value
        for _ in range(x_dim):
            allpointslist.append(pd.to_numeric("{:.5f}".format(x_value)))
            allpointslist.append(pd.to_numeric("{:.5f}".format(y_value)))
            allpointslist.append(0)
            x_value += x_step
        y_value += y_step
else:
    print("3D file")
    z_value = z_start_value
    for _ in range(z_dim):
        y_value = y_start_value
        for _ in range(y_dim):
            x_value = x_start_value
            for _ in range(x_dim):
                allpointslist.append(pd.to_numeric("{:.5f}".format(x_value)))
                allpointslist.append(pd.to_numeric("{:.5f}".format(y_value)))
                allpointslist.append(pd.to_numeric("{:.5f}".format(z_value)))
                x_value += x_step
            y_value += y_step
        z_value += z_step


        
x = int(len(allpointslist)/3)
# print(x)

allpointsarray = np.array(allpointslist)
allpointsarray = allpointsarray.reshape(x,3)
allpointsdf = pd.DataFrame(allpointsarray)
allpointsdf.to_csv("AllPoints3D.csv", header=False, index=False, float_format='%.5f')
# print(allpointsdf)

This section goes through the TexGen file that we want to recreate. It creates a csv file for each of the important bits of information that we will want to reference later. It wouldn't be hard to make it so we just use dataframes instead of csv files, but I find that they're potentially convenient for troubleshooting if necessary so I never felt the need to change that.

In [ ]:
yarns = 1

with open(InputFileName,"r") as infile:
    with open("AllNodes.csv",'w') as AllNodes:
        for line in infile:
            if line.startswith("*Node"):
                for line in infile:
                    with open("AllElements.csv",'w') as AllElements:
                        if line.startswith("*Element"):
                            for line in infile:
                                if line.startswith("*************"):
                                    with open("AllElementSets.csv",'w') as AllElementSets:
                                        for line in infile:
                                            if line.startswith("*ElSet, ElSet=Yarn0"):
                                                for line in infile:
                                                    if line.startswith("*ElSet"):
                                                        yarns += 1
                                                    if line.startswith("****"):
                                                        break
                                                    else:
                                                        AllElementSets.write(line)
                                            # else:
                                            #     print(line)
                                                    
                                else:
                                    AllElements.write(line)
                        else:
                            AllNodes.write(line)

        

We read these two csvs into dataframes. Elements are our shapes that we will treat as hulls that form the fibers. Nodes are just any xyz coordinate that the TexGen file defines.

In [ ]:
AllElements = pd.read_csv("AllElements.csv", header=None)
AllNodes = pd.read_csv("AllNodes.csv", header=None)

We create a dictionary that holds all the different yarns based on the number defined in the TexGen file. The print line shows the count for number of yarns (starting at 0). This also attributes each element to its respective yarn number.

In [ ]:
d = {}


with open("AllElementSets.csv", "r") as AllElementSets:
    for yarn in range(yarns):
        print(yarn)
        d["Yarn{0}".format(yarn)] = []
        for rec in csv.reader(AllElementSets, delimiter=','):
            if '*ElSet' in rec:
                break
            else:
                newrec =  list(map(int,rec))
                for item in newrec:
                    d["Yarn{0}".format(yarn)].append(item)


We are creating a copy of the points in our box so that we have them in a testable format. Then we add our "inHull" column to the dataframe that defines all the points in our box. This column will essentially hold our grain/fiber number.

In [ ]:
tested = allpointsdf[[0,1,2]].to_numpy()
allpointsdf.insert(3, "inHull", 0)

This defines the in_hull function which we use to check if points are inside of a "hull". The TexGen file saves each fiber as a combination of multiple different elements based on a number of nodes which are defined with a counter identifier and xyz coordinates. We count each element as a hull and then check whether a given point is in that hull, then we can assign it to a fiber if it's in a hull that is part of a specific fiber.

In [ ]:
def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

This is where the magic happens and is typically the longest running section, especially for larger 3D files. We are going through each point defined in our box and checking against each element to determine if it is inside any of them. Since we attributed each element to a specific yarn, we then can define our "inHull" column as the respective fiber number. 

In [ ]:
hulldict = {}

for yarn in range(yarns):
    listofpoints = []
    for element in d["Yarn{0}".format(yarn)]:
        eles = AllElements.loc[element-1,1:]
        for each in eles:
            for i in AllNodes.loc[each-1,1:]:
                listofpoints.append(i)
    length = int(len(listofpoints)/3)
    arrayofpoints = np.array(listofpoints)
    
    
    newlength = int(length/10)


    arrayofpoints = arrayofpoints.reshape(length,3)
    newarrayofpoints = arrayofpoints.reshape(newlength,10,3)
    

    for testplot in newarrayofpoints:
        allpointsdf['inHull'] = allpointsdf['inHull'].mask(allpointsdf['inHull']==0,in_hull(tested,testplot).astype(int)*(yarn+1))


Quick test to make sure our code worked. This just shows a graphical representation of what our box looks like now that we have checked each point in the box against the elements in our TexGen file.

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(projection='3d')

x = allpointsdf.iloc[:,0]
y = allpointsdf.iloc[:,1]
z = allpointsdf.iloc[:,2]

ax.scatter(x, y, z, c=allpointsdf.loc[:,"inHull"], alpha=0.01, cmap='viridis')

plt.show()

## Basic 2D File Generation
This takes a slice from the 3D model and uses it for the 2D file example. It slices through the middle of the Y axis and then flips it so the Z axis is treated as the Y axis for Macaw.

Again create our new dataframe to work with. Then we take a 2D slice of our box. This example pulls an XZ slice from the middle of Y axis of our box. This was just chosen because it's the most interesting for this sample.

In [ ]:
twoDebsddf = allpointsdf.copy(deep=True)
twoDebsddf = twoDebsddf[twoDebsddf[1] == pd.to_numeric("{:.5f}".format(y_start_value+(y_dim/2-1)*y_step))]

Quick graph to show our 2D slice.

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(projection='3d')

x = twoDebsddf.iloc[:,0]
y = twoDebsddf.iloc[:,1]
z = twoDebsddf.iloc[:,2]

ax.scatter(x, y, z, c=twoDebsddf.loc[:,"inHull"], alpha=0.1, cmap='viridis')

plt.show()

Scaling changes.

In [ ]:
twoDebsddf[0] = twoDebsddf[0] * NonDimConv
twoDebsddf[1] = twoDebsddf[1] * NonDimConv
twoDebsddf[2] = twoDebsddf[2] * NonDimConv

Quick graph to show how we actually want this to look in Macaw since it prefers XY systems instead of XZ systems.

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(projection='3d')

x = twoDebsddf.iloc[:,0]
y = twoDebsddf.iloc[:,2]
z = twoDebsddf.iloc[:,1]

ax.scatter(x, y, z, c=twoDebsddf.loc[:,"inHull"], alpha=0.1, cmap='viridis')

plt.show()

Adjusts our dataframe to match the graph from above. Then we add our phase values and extra columns required for the EBSD Reader file format.

In [ ]:
twoDebsddf= twoDebsddf.drop(columns=[1])
twoDebsddf = twoDebsddf.rename(columns={0: "x", 2: "y"})
twoDebsddf['phase'] = twoDebsddf['inHull'].mask(twoDebsddf['inHull'] >0, 2)
twoDebsddf['phase'] = twoDebsddf['phase'].mask(twoDebsddf['phase'] <1, 1)
twoDebsddf.insert(2, "z", 0)
twoDebsddf.insert(0, "phi1", 0)
twoDebsddf.insert(1, "phi2", 0)
twoDebsddf.insert(2, "phi3", 0)
twoDebsddf.insert(8, "symmetry", 43)

Creates our 2D EBSD file that we can import into Macaw.

In [ ]:
with open(OutputFileNameBase+"2D_ebsd.txt", "w") as file:
    file.write(f'# X_STEP: {x_step* NonDimConv}\n')
    file.write(f'# Y_STEP: {z_step* NonDimConv}\n')
    file.write(f'# Z_STEP: 0\n')
    file.write('#\n')
    file.write(f'# X_MIN: {x_min* NonDimConv}\n')
    file.write(f'# Y_MIN: {z_min* NonDimConv}\n')
    file.write(f'# Z_MIN: 0\n')
    file.write('#\n')
    file.write(f'# X_MAX: {x_max* NonDimConv}\n')
    file.write(f'# Y_MAX: {z_max* NonDimConv}\n')
    file.write(f'# Z_MAX: 0\n')
    file.write('#\n')
    file.write(f'# X_DIM: {x_dim}\n')
    file.write(f'# Y_DIM: {z_dim}\n')
    file.write(f'# Z_DIM: 0\n')
    file.write('#\n')
    file.write('# Phase 1: Matrix\n')
    file.write('# Features_1: 1\n')
    file.write('# Phase 2: Yarn\n')
    file.write(f'# Features_2: {yarns}\n')  
    file.write('#\n')
    file.write('# phi1 PHI phi2 x y z FeatureId PhaseId Symmetry')
    file.write('\n')
    
twoDebsddf.to_csv(OutputFileNameBase+"2D_ebsd.txt", mode='a', sep =' ', index=False, header=False, float_format='%.5f')

The following code creates an EBSD file used for the BoxTest example. This example can be used for quick testing on a much smaller scale.

In [ ]:
import io

with open(OutputFileNameBase+"2D_ebsd.txt", "r") as file:
    lines = file.readlines()
    data_lines = lines[21:]
    df_2d = pd.read_csv(io.StringIO(''.join(data_lines)), sep=' ', header=0)
    df_2d.columns = df_2d.columns[1:].tolist() + [df_2d.columns[0]]
    df_2d = df_2d.drop(columns=df_2d.columns[-1])
    file.close()
    # lines = file.readlines()
    # print(lines[21:23])  # Print the first 10 lines to verify the content
print(df_2d.head())  # Display the first few rows of the DataFrame to verify the content
print("Number of rows in df_2d:", len(df_2d))
df_2d = df_2d[df_2d['x'] <= 12000]
print(df_2d.head())
print("Number of rows in df_2d:", len(df_2d))
df_2d = df_2d[df_2d['y'] >= 100000]
print(df_2d.head())
print("Number of rows in df_2d:", len(df_2d))
print("Unique values in x:", df_2d['x'].nunique())
print("Unique values in z:", df_2d['y'].nunique())


with open("SimpleBoxTest_EBSD.txt", "w") as file:
    file.write(f'# X_STEP: {x_step* NonDimConv}\n')
    file.write(f'# Y_STEP: {z_step* NonDimConv}\n')
    file.write(f'# Z_STEP: 0\n')
    file.write('#\n')
    file.write(f'# X_MIN: {x_min}\n')
    file.write(f'# Y_MIN: 100000\n')
    file.write(f'# Z_MIN: 0\n')
    file.write('#\n')
    file.write(f'# X_MAX: {x_max* NonDimConv}\n')
    file.write(f'# Y_MAX: {z_max* NonDimConv}\n')
    file.write(f'# Z_MAX: 0\n')
    file.write('#\n')
    file.write(f'# X_DIM: {df_2d["x"].nunique()}\n')
    file.write(f'# Y_DIM: {df_2d["y"].nunique()}\n')
    file.write(f'# Z_DIM: 0\n')
    file.write('#\n')
    file.write('# Phase 1: Matrix\n')
    file.write('# Features_1: 1\n')
    file.write('# Phase 2: Yarn\n')
    file.write(f'# Features_2: 2\n')  
    file.write('#\n')
    file.write('# phi1 PHI phi2 x y z FeatureId PhaseId Symmetry')
    file.write('\n')
    
df_2d.to_csv('SimpleBoxTest_EBSD.txt', mode='a', sep =' ', index=False, header=False, float_format='%.5f')

## Basic 3D EBSD File Generation

Since TexGen always outputs a 3D file, we need to run this before creating any of the other file types. This will create our general 3D EBSD file.

This entire script creates multiple different EBSD files that we can bring into Macaw. Since we will be doing edits to each to get them into the desired format for Macaw, I make a copy from our master box dataframe before going into each. There will be similar code in each file generation section, but it's typically on different data or in a different order to get the EBSD file that we want to load into Macaw.

In [ ]:
ebsddf = allpointsdf.copy(deep=True)

Macaw has a different scale factor relative to the TexGen file. This adjusts the scale factor to be inline for Macaw.

In [ ]:
ebsddf[0] = ebsddf[0] * NonDimConv
ebsddf[1] = ebsddf[1] * NonDimConv
ebsddf[2] = ebsddf[2] * NonDimConv

This checks if a fiber is defined for each point in our box so that it can define a different phase for fibers vs matrix.

In [ ]:
ebsddf['phase'] = ebsddf['inHull'].mask(ebsddf['inHull'] >0, 2)
ebsddf['phase'] = ebsddf['phase'].mask(ebsddf['phase'] <1, 1)
ebsddf.insert(0, "phi1", 0)
ebsddf.insert(1, "phi2", 0)
ebsddf.insert(2, "phi3", 0)
ebsddf.insert(8, "symmetry", 43)

This creates the EBSD text file that we will import into Macaw.

In [ ]:
with open(OutputFileNameBase+"3D_ebsd.txt", "w") as file:
    file.write(f'# X_STEP: {x_step* NonDimConv}\n')
    file.write(f'# Y_STEP: {y_step* NonDimConv}\n')
    file.write(f'# Z_STEP: {z_step* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_MIN: {x_min* NonDimConv}\n')
    file.write(f'# Y_MIN: {y_min* NonDimConv}\n')
    file.write(f'# Z_MIN: {z_min* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_MAX: {x_max* NonDimConv}\n')
    file.write(f'# Y_MAX: {y_max* NonDimConv}\n')
    file.write(f'# Z_MAX: {z_max* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_DIM: {x_dim}\n')
    file.write(f'# Y_DIM: {y_dim}\n')
    file.write(f'# Z_DIM: {z_dim}\n')
    file.write('#\n')
    file.write('# Phase 1: Matrix\n')
    file.write('# Features_1: 1\n')
    file.write('# Phase 2: Yarn\n')
    file.write(f'# Features_2: {yarns}\n')  
    file.write('#\n')
    file.write('# phi1 PHI phi2 x y z FeatureId PhaseId Symmetry')
    file.write('\n')
    
ebsddf.to_csv(OutputFileNameBase+"3D_ebsd.txt", mode='a', sep =' ', index=False, header=False, float_format='%.5f')

## WIP 3D EBSD File with Voids
This section produces an EBSD file for a 3D system and implements some basic voids to the system as well. Using this is not yet implemented in Macaw.

Similar to before. We create a copy of our dataframe and then define our phase based on if there is an assigned fiber or not.

In [ ]:
Voidsdf = allpointsdf.copy(deep=True)
Voidsdf['phase'] = Voidsdf['inHull'].mask(Voidsdf['inHull'] >0, 2)
Voidsdf['phase'] = Voidsdf['phase'].mask(Voidsdf['phase'] <1, 1)


We create 50 voids with a radius of 0.1 around a center point chosen at random in our box. Each point in the void is checked against the phase to determine if that point is already defined as part of a fiber. If it is part of a fiber, that point in the void is ignored so we only overwrite points in our box that are defined as matrix. 

In [ ]:
void_radius = .1

for voidNum in range(50):
    void = np.random.rand(3)

    void[0] = void[0]*x_step*x_dim + x_min
    void[1] = void[1]*y_step*y_dim + y_min
    void[2] = void[2]*z_step*z_dim + z_min

    Voidsdf['phase'] = Voidsdf['phase'].mask( (((abs(Voidsdf[0]-void[0]) + abs(Voidsdf[1]-void[1]) + abs(Voidsdf[2]-void[2])) < void_radius) &  (Voidsdf['phase']==1)), 3)

Voidsdf['inHull'] = Voidsdf['inHull'].mask(Voidsdf['phase'] >2, yarns+1)

A quick visualization of our voids. As a note, the voids may look non-spherical due to the scale bars set automatically by the graph. The matrix isn't shown.

In [ ]:
dryfiberdfvoid = Voidsdf.copy(deep=True)
dryfiberdfvoid = dryfiberdfvoid[dryfiberdfvoid.inHull >0 ]

fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(projection='3d')

x = dryfiberdfvoid.iloc[:,0]
y = dryfiberdfvoid.iloc[:,1]
z = dryfiberdfvoid.iloc[:,2]

ax.scatter(x, y, z, c=dryfiberdfvoid.loc[:,"inHull"], cmap='viridis')

plt.show()

Next three sections add columns, adjust the scale to fit Macaw, and write the EBSD file that we'll import into Macaw.

In [ ]:
Voidsdf.insert(0, "phi1", 0)
Voidsdf.insert(1, "phi2", 0)
Voidsdf.insert(2, "phi3", 0)
Voidsdf.insert(8, "symmetry", 43)

In [ ]:
Voidsdf[0] = Voidsdf[0] * NonDimConv
Voidsdf[1] = Voidsdf[1] * NonDimConv
Voidsdf[2] = Voidsdf[2] * NonDimConv

In [ ]:
with open(OutputFileNameBase+"withVoids_ebsd.txt", "w") as file:
    file.write(f'# X_STEP: {x_step* NonDimConv}\n')
    file.write(f'# Y_STEP: {y_step* NonDimConv}\n')
    file.write(f'# Z_STEP: {z_step* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_MIN: {x_min* NonDimConv}\n')
    file.write(f'# Y_MIN: {y_min* NonDimConv}\n')
    file.write(f'# Z_MIN: {z_min* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_MAX: {x_max* NonDimConv}\n')
    file.write(f'# Y_MAX: {y_max* NonDimConv}\n')
    file.write(f'# Z_MAX: {z_max* NonDimConv}\n')
    file.write('#\n')
    file.write(f'# X_DIM: {x_dim}\n')
    file.write(f'# Y_DIM: {y_dim}\n')
    file.write(f'# Z_DIM: {z_dim}\n')
    file.write('#\n')
    file.write('# Phase 1: Matrix\n')
    file.write('# Features_1: 1\n')
    file.write('# Phase 2: Yarn\n')
    file.write(f'# Features_2: {yarns}\n')
    file.write('# Phase 3: Voids\n')
    file.write('# Features_2: 1\n')
    file.write('#\n')
    file.write('# phi1 PHI phi2 x y z FeatureId PhaseId Symmetry')
    file.write('\n')
    
Voidsdf.to_csv(OutputFileNameBase+"withVoids_ebsd.txt", mode='a', sep =' ', index=False, header=False, float_format='%.5f')

## WIP 2D EBSD file with Voids
This section produces an EBSD file for a 2D system and implements some basic voids to the system as well. Using this is not yet implemented in Macaw.

Create our new dataframe copy and define our phases.

In [ ]:
twoDebsddfwithVoids = allpointsdf.copy(deep=True)
twoDebsddfwithVoids['phase'] = twoDebsddfwithVoids['inHull'].mask(twoDebsddfwithVoids['inHull'] >0, 2)
twoDebsddfwithVoids['phase'] = twoDebsddfwithVoids['phase'].mask(twoDebsddfwithVoids['phase'] <1, 1)


Creates voids similar to our 3D model, but doesn't check for the Y coordinate when defining the void. This means that it will create the void across the entire Y direction as long as it's within the radius in the XZ directions and not already defined as fiber.

In [ ]:
void_radius = .05

for voidNum in range(50):
    void = np.random.rand(3)

    void[0] = void[0]*x_step*x_dim + x_min
    void[1] = void[1]*y_step*y_dim + y_min
    void[2] = void[2]*z_step*z_dim + z_min

    twoDebsddfwithVoids['phase'] = twoDebsddfwithVoids['phase'].mask( (((abs(twoDebsddfwithVoids[0]-void[0]) + abs(twoDebsddfwithVoids[2]-void[2])) < void_radius) &  (twoDebsddfwithVoids['phase']==1)), 3)

twoDebsddfwithVoids['inHull'] = twoDebsddfwithVoids['inHull'].mask(twoDebsddfwithVoids['phase'] >2, yarns+1)

Takes the 2D slice from our 3D box, again from the middle of our Y axis.

In [ ]:
twoDebsddfwithVoids = twoDebsddfwithVoids[twoDebsddfwithVoids[1] == pd.to_numeric("{:.5f}".format(y_start_value+(y_dim/2-1)*y_step))]

Graph showing our 2D slice.

In [ ]:
fig = plt.figure(figsize=(12, 9))
ax = fig.add_subplot(projection='3d')

x = twoDebsddfwithVoids.iloc[:,0]
y = twoDebsddfwithVoids.iloc[:,1]
z = twoDebsddfwithVoids.iloc[:,2]

ax.scatter(x, y, z, c=twoDebsddfwithVoids.loc[:,"inHull"], alpha=0.1, cmap='viridis')

plt.show()

Next 3 sections change the scaling, flip to an XY system, add columns, and export our EBSD file that we can import into Macaw.

In [ ]:
twoDebsddfwithVoids[0] = twoDebsddfwithVoids[0] * NonDimConv
twoDebsddfwithVoids[1] = twoDebsddfwithVoids[1] * NonDimConv
twoDebsddfwithVoids[2] = twoDebsddfwithVoids[2] * NonDimConv

In [ ]:
twoDebsddfwithVoids= twoDebsddfwithVoids.drop(columns=[1])
twoDebsddfwithVoids = twoDebsddfwithVoids.rename(columns={0: "x", 2: "y"})
twoDebsddfwithVoids.insert(2, "z", 0)
twoDebsddfwithVoids.insert(0, "phi1", 0)
twoDebsddfwithVoids.insert(1, "phi2", 0)
twoDebsddfwithVoids.insert(2, "phi3", 0)
twoDebsddfwithVoids.insert(8, "symmetry", 43)

In [ ]:
with open(OutputFileNameBase+"CharOx_2D_ebsd.txt", "w") as file:
    file.write(f'# X_STEP: {x_step* NonDimConv}\n')
    file.write(f'# Y_STEP: {z_step* NonDimConv}\n')
    file.write(f'# Z_STEP: 0\n')
    file.write('#\n')
    file.write(f'# X_MIN: {x_min* NonDimConv}\n')
    file.write(f'# Y_MIN: {z_min* NonDimConv}\n')
    file.write(f'# Z_MIN: 0\n')
    file.write('#\n')
    file.write(f'# X_MAX: {x_max* NonDimConv}\n')
    file.write(f'# Y_MAX: {z_max* NonDimConv}\n')
    file.write(f'# Z_MAX: 0\n')
    file.write('#\n')
    file.write(f'# X_DIM: {x_dim}\n')
    file.write(f'# Y_DIM: {z_dim}\n')
    file.write(f'# Z_DIM: 0\n')
    file.write('#\n')
    file.write('# Phase 1: Matrix\n')
    file.write('# Features_1: 1\n')
    file.write('# Phase 2: Yarn\n')
    file.write(f'# Features_2: {yarns}\n')
    file.write('# Phase 3: Voids\n')
    file.write('# Features_2: 1\n')
    file.write('#\n')
    file.write('# phi1 PHI phi2 x y z FeatureId PhaseId Symmetry')
    file.write('\n')
    
twoDebsddfwithVoids.to_csv(OutputFileNameBase+"CharOx_2D_ebsd.txt", mode='a', sep =' ', index=False, header=False, float_format='%.5f')